In [0]:
%run ./Shared_Functions

In [0]:
# COMMAND ----------
# 0. SETUP & UTILITIES
# MAGIC %run ./Shared_Functions

# COMMAND ----------
import pyspark.sql.functions as F

In [0]:
# GET RUN ID 
try:
    # This magic line grabs the unique ID of this current execution
    run_id = dbutils.notebook.entry_point.getDbutils().notebook().getContext().runId().get()
except:
    run_id = "manual_run_gold"

In [0]:
# ==========================================
# TASK 1: THE "USER 360" ONE BIG TABLE (With AI Metadata)
# ==========================================
job_name_obt = "Gold_One_Big_Table"
try:
    print(f">>> [START] {job_name_obt}...")

    # 1. READ & FILTER
    u = spark.read.table("ecomm_data_project.silver.users").filter("is_current = true").alias("u")
    b = spark.read.table("ecomm_data_project.silver.buyers").alias("b")
    s = spark.read.table("ecomm_data_project.silver.sellers").alias("s")
    c = spark.read.table("ecomm_data_project.silver.countries").alias("c")

    # 2. JOIN LOGIC
    gold_final_df = u \
        .join(c, u.countrycode == c.country, "left") \
        .join(b, c.country == b.country, "left") \
        .join(s, c.country == s.country, "left") \
        .select(
            F.col("u.identifierHash").alias("User_ID"),
            F.col("u.countrycode").alias("Country"),
            F.col("u.gender").alias("User_Gender"),
            F.col("u.productsSold").alias("User_ProductsSold"),
            F.col("u.productsWished").alias("User_ProductsWished"),
            F.col("u.account_age_years").alias("User_AccountAge"),
            F.col("s.nbsellers").alias("Country_TotalSellers"),
            F.col("b.buyers").alias("Country_TotalBuyers"),
            F.col("b.female_to_male_ratio").alias("Country_BuyerRatio")
        )

    # 3. WRITE WITH METADATA (The MNC Way)
    # We use 'overwrite' to ensure the schema and comments are fresh
    gold_final_df.write.format("delta") \
        .mode("overwrite") \
        .saveAsTable("ecomm_data_project.gold.ecom_one_big_table")

    # 4. SET TABLE & COLUMN COMMENTS FOR THE AI AGENT
    # This acts as the 'Documentation' the LLM reads to understand your data
    spark.sql("COMMENT ON TABLE ecomm_data_project.gold.ecom_one_big_table IS 'Primary user 360 table containing demographics and country-level sales metrics for e-commerce analysis.'")
    
    spark.sql("ALTER TABLE ecomm_data_project.gold.ecom_one_big_table ALTER COLUMN User_ProductsSold COMMENT 'Total count of successful sales made by this user.'")
    spark.sql("ALTER TABLE ecomm_data_project.gold.ecom_one_big_table ALTER COLUMN Country_BuyerRatio COMMENT 'The ratio of female to male buyers in the users country. Used for gender-based market analysis.'")
    spark.sql("ALTER TABLE ecomm_data_project.gold.ecom_one_big_table ALTER COLUMN User_AccountAge COMMENT 'The number of years the user has been registered on the platform.'")

    log_pipeline_run(run_id, job_name_obt, "Gold", "SUCCESS", gold_final_df.count())
    print(f">>> [SUCCESS] {job_name_obt} complete with AI Metadata.")

except Exception as e:
    log_pipeline_run(run_id, job_name_obt, "Gold", "FAILED", 0, str(e))
    raise e

In [0]:
# TASK 2: USER DEMOGRAPHICS AGGREGATION
# ==========================================
job_name_demo = "Gold_User_Demographics"
try:
    print(f">>> [START] {job_name_demo}...")
    
    # Aggregate only active users
    demo_df = spark.sql("""
        SELECT 
            countrycode,
            gender,
            COUNT(identifierHash) as total_users,
            AVG(account_age_years) as avg_account_age
        FROM ecomm_data_project.silver.users
        WHERE is_current = true
        GROUP BY countrycode, gender
    """)
    
    demo_df.write.format("delta").mode("overwrite").saveAsTable("ecomm_data_project.gold.user_demographics")
    
    # Log Success
    log_pipeline_run(run_id, job_name_demo, "Gold", "SUCCESS", demo_df.count())
    print(f">>> [SUCCESS] {job_name_demo} Refreshed.")

except Exception as e:
    log_pipeline_run(run_id, job_name_demo, "Gold", "FAILED", 0, str(e))
    print(f"!!! [ERROR] {job_name_demo} failed: {e}")